In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import regularizers

# Define constants
NUM_CLASSES = 4  # Number of classes (correct, partially correct, incorrect, none)
IMG_WIDTH, IMG_HEIGHT = 224, 224  # Input image dimensions for ResNet50
BATCH_SIZE = 32
EPOCHS = 25

# Load ResNet50 pre-trained model (include_top=False to exclude the classification layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

# Flatten the output from the base model
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = Dropout(0.5)(x)  # Adjusted dropout rate
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Implement a learning rate schedule
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Create data generators for training and validation with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    '/home/adarsh/Desktop/yoga/train/',
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

validation_generator = validation_datagen.flow_from_directory(
    '/home/adarsh/Desktop/yoga/test/',
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Define early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with early stopping and learning rate schedule
model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // BATCH_SIZE,
    callbacks=[early_stop, lr_schedule]  # Add early stopping and LR schedule to the callbacks
)

# Save the model for later use
model.save('pose_classification_model_resnet50.h5')


2023-09-24 00:56:40.453322: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-24 00:56:40.501402: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-24 00:56:40.502361: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-24 00:56:41.702427: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-24 00:56:43.382436: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Do

Found 24855 images belonging to 4 classes.
Found 5636 images belonging to 4 classes.
Epoch 1/25
776/776 [==============================] - 3419s 4s/step - loss: 1.1968 - accuracy: 0.8989 - val_loss: 2.4530 - val_accuracy: 0.0648 - lr: 0.0010
Epoch 2/25
776/776 [==============================] - 3431s 4s/step - loss: 0.1492 - accuracy: 0.9652 - val_loss: 0.8939 - val_accuracy: 0.7951 - lr: 0.0010
Epoch 3/25
776/776 [==============================] - 3434s 4s/step - loss: 0.1294 - accuracy: 0.9695 - val_loss: 0.5230 - val_accuracy: 0.8365 - lr: 0.0010
Epoch 4/25
776/776 [==============================] - 3432s 4s/step - loss: 0.0823 - accuracy: 0.9813 - val_loss: 0.2111 - val_accuracy: 0.9308 - lr: 0.0010
Epoch 5/25
776/776 [==============================] - 3448s 4s/step - loss: 0.1160 - accuracy: 0.9733 - val_loss: 0.2855 - val_accuracy: 0.9103 - lr: 0.0010
Epoch 6/25
776/776 [==============================] - 3447s 4s/step - loss: 0.0623 - accuracy: 0.9865 - val_loss: 0.3840 - val_acc

/home/adarsh/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
